Run these lines once:

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} scipy

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/lavanyasingh/anaconda3

  added / updated specs:
    - scipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scipy-1.3.2                |   py36h1410ff5_0        12.5 MB
    ------------------------------------------------------------
                                           Total:        12.5 MB

The following NEW packages will be INSTALLED:

  scipy              pkgs/main/osx-64::scipy-1.3.2-py36h1410ff5_0



scipy-1.3.2          | 12.5 MB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Read data from csv into pandas dataframe

In [2]:
import pandas as pd
import csv

data = {"source":[], "dest_ip":[], "dest_owner":[], "type":[], "OS": [], "length":[], "app":[]}
with open("parsed.csv", "r") as inf:
    reader = csv.reader(inf, delimiter=',', quotechar = '"')
    next(reader)
    for row in reader:
        data["source"].append(row[0])
        data["dest_ip"].append(row[1])
        data["dest_owner"].append(row[2][row[2].find("-") + 2:])
        data["type"].append(row[3])
        data["OS"].append(row[4])
        data["length"].append(row[5])
        data["app"].append(row[6])
df = pd.DataFrame(data)


Top 15 most frequent providers (based on packet count)

In [3]:
top_fifteen = df[df.dest_owner != ""]["dest_owner"].value_counts()[0:15]
print(top_fifteen)

Google LLC, US                                                  43258
Comcast Cable Communications, LLC, US                           35739
2 - Amazon.com, Inc., US                                        13112
RODUCTION - Roblox, US                                           9805
SN1, US                                                          5092
S - Akamai Technologies, Inc., US                                4564
MCI Communications Services, Inc. d/b/a Verizon Business, US     4458
Facebook, Inc., US                                               2437
Highwinds Network Group, Inc., US                                2293
ES - Amazon.com, Inc., US                                        2185
Fastly, US                                                       1650
NGINEERING - Apple Inc., US                                       437
Square, Inc., US                                                  435
ORP-MSN-AS-BLOCK - Microsoft Corporation, US                      229
MoPub, Inc., US     

Previous analysis does not take into account the differences in number of packets for each app (this is why roblox is so high). 
Let's rank providers based on the proportion of packets captured for each app. 


Score for a given provider is given by  $$Score = \frac{1}{N} \sum_{n=1}^{N} \frac{P_n}{T_n}$$

Where $N$ is the number of apps, $P_n$ is the number of outgoing packets for app $n$ handled by the provider, and $T_n$ is the total number of outgoing packets for app $n$. Thus, the maximum possible score (all packets handled for all apps) is 1.

In [19]:
def get_scores(df):
    owner_dict = {key:0 for key in df.dest_owner.unique()}
    df = df[df.dest_owner != ""]
    
    
    for app in df.app.unique():
        perc = df[df.app == app]["dest_owner"].value_counts()
        total = len(df[df.app == app])
        perc = {key: float(value/total) for key, value in perc.items()}
        for key,value in perc.items(): owner_dict[key] += value
        
    apps = len(df.app.unique())
    score = {key: float(value)/apps for key,value in owner_dict.items()}
    scores = pd.DataFrame(sorted(score.items(), key=lambda x: x[1], reverse=True), columns = ["Provider", "Score"])
    scores["% Traffic"] = scores["Provider"].map(lambda x: len(df[df.dest_owner == x])/len(df)*100)
    return scores

print(get_scores(df).head(15))

                                             Provider     Score  % Traffic
0                                      Google LLC, US  0.442675  34.162020
1                            2 - Amazon.com, Inc., US  0.159281  10.354903
2               Comcast Cable Communications, LLC, US  0.154884  28.224061
3                   S - Akamai Technologies, Inc., US  0.047973   3.604315
4                                  Facebook, Inc., US  0.035142   1.924565
5                           ES - Amazon.com, Inc., US  0.031029   1.725554
6                                             SN1, US  0.025295   4.021291
7   MCI Communications Services, Inc. d/b/a Verizo...  0.022638   3.520604
8                                          Fastly, US  0.020822   1.303050
9                         NGINEERING - Apple Inc., US  0.017200   0.345111
10                             RODUCTION - Roblox, US  0.015031   7.743275
11                  Highwinds Network Group, Inc., US  0.009439   1.810845
12                       

Let's repeat the same thing for ios.

In [20]:
print(get_scores(df[df.OS == "ios"]).head(15))

                                        Provider     Score  % Traffic
0                                 Google LLC, US  0.243643  33.850583
1                       2 - Amazon.com, Inc., US  0.194711  21.492768
2                             Facebook, Inc., US  0.153152   3.917989
3                    NGINEERING - Apple Inc., US  0.110250   1.534195
4          Comcast Cable Communications, LLC, US  0.089188  20.264008
5              S - Akamai Technologies, Inc., US  0.075360   6.445724
6                                     Fastly, US  0.042176   3.163179
7                      ES - Amazon.com, Inc., US  0.030224   3.198287
8                               Square, Inc., US  0.029450   1.105884
9                         USTIN - Apple Inc., US  0.011738   0.165005
10             Highwinds Network Group, Inc., US  0.010921   2.871788
11                          Cloudflare, Inc., US  0.002293   0.298413
12  ORP-MSN-AS-BLOCK - Microsoft Corporation, US  0.002111   0.628423
13                  

And for android.

In [21]:
print(get_scores(df[df.OS == "android"]).head(15))

                                             Provider     Score  % Traffic
0                                      Google LLC, US  0.525805  34.252410
1                            2 - Amazon.com, Inc., US  0.136185   7.122333
2               Comcast Cable Communications, LLC, US  0.123039  30.534328
3                   S - Akamai Technologies, Inc., US  0.050349   2.779646
4                                             SN1, US  0.028707   5.188400
5                           ES - Amazon.com, Inc., US  0.027426   1.298119
6   MCI Communications Services, Inc. d/b/a Verizo...  0.024967   4.509792
7                                  Facebook, Inc., US  0.024638   1.346009
8                                          Fastly, US  0.016828   0.763180
9                              RODUCTION - Roblox, US  0.016701   9.990626
10                  Highwinds Network Group, Inc., US  0.011831   1.502924
11                                    MoPub, Inc., US  0.003295   0.173218
12                       

android talks to google WAY more -> indicative of background/non-app related traffic???

Now let's analyze the distribution of packet providers. Let's calculate Shannon's entropy for each app. This value will be highest if providers are evenly distributed. 

In [25]:
from scipy.stats import entropy

app_dict = {app:None for app in df.app.unique()}
for app in df.app.unique():
    counts = [len(df[df["app"] == app][df["dest_owner"] == dest_owner]) for dest_owner in df.dest_owner.unique()]
    app_dict[app] = entropy(counts, base=None)
app_entropy = pd.DataFrame(sorted(app_dict.items(), key=lambda x: x[1], reverse=True), columns = ['App', 'Entropy'])
app_entropy["CDN Count"] = [len(df[df.app == app].dest_owner.unique()) for app in app_entropy['App']]
app_entropy["Top CDN"] = [df[df.app == app]['dest_owner'].value_counts().idxmax() for app in app_entropy['App']]
print(app_entropy)

/Users/lavanyasingh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


               App   Entropy  CDN Count                                Top CDN
0       forgeahead  2.200191         23      S - Akamai Technologies, Inc., US
1           tinder  1.914604         10                         Google LLC, US
2          luckygo  1.560211         13      S - Akamai Technologies, Inc., US
3           arrive  1.425340          7                         Google LLC, US
4             yolo  1.368666         11                         Google LLC, US
5            venmo  1.367446         13               2 - Amazon.com, Inc., US
6          walmart  1.158229          9  Comcast Cable Communications, LLC, US
7          cashapp  1.127729          8                         Google LLC, US
8           tiktok  1.021691          9                                SN1, US
9        messenger  0.950322         11                         Google LLC, US
10  amazonshopping  0.863552          5  Comcast Cable Communications, LLC, US
11          roblox  0.750157          9  Comcast Cab

Now let's do some entropy analysis by category. I'm going to manually input categories :(

In [54]:
categories = {"forgeahead":"games", "tinder":"lifestyle", "luckygo":"lifestyle", "arrive":"shopping", "yolo":"social", 
              "venmo":"finance", "walmart":"shopping", "cashapp":"finance", "tiktok":"social", "messenger":"social", 
             "amazonshopping":"shopping", "roblox":"games", "googlehome":"lifestyle", "amazon":"shopping", "disneyplus":"entertainment", 
             "messengerkids":"kids", "baseline":"control", "snapchat":"social", "youtubekids":"kids", "androidauto":"auto"}
app_entropy["category"] = [categories[app] for app in app_entropy.App.unique()]
# making the provider names shorter lol
app_entropy["Top CDN"] = [provider[0:15] for provider in app_entropy["Top CDN"]]
print(app_entropy)

               App   Entropy  CDN Count          Top CDN       category
0       forgeahead  2.200191         23  S - Akamai Tech          games
1           tinder  1.914604         10   Google LLC, US      lifestyle
2          luckygo  1.560211         13  S - Akamai Tech      lifestyle
3           arrive  1.425340          7   Google LLC, US       shopping
4             yolo  1.368666         11   Google LLC, US         social
5            venmo  1.367446         13  2 - Amazon.com,        finance
6          walmart  1.158229          9  Comcast Cable C       shopping
7          cashapp  1.127729          8   Google LLC, US        finance
8           tiktok  1.021691          9          SN1, US         social
9        messenger  0.950322         11   Google LLC, US         social
10  amazonshopping  0.863552          5  Comcast Cable C       shopping
11          roblox  0.750157          9  Comcast Cable C          games
12      googlehome  0.697491          4   Google LLC, US      li

Now let's get entropy by category.

In [55]:
# a little maintenance
df["category"] = [categories[app] for app in df.app]
df["dest_owner"] = [provider[0:15] for provider in df.dest_owner]

cat_dict = {category:None for category in categories.values()}
for category in df.category.unique():
    counts = [len(df[df["category"] == category][df["dest_owner"] == dest_owner]) for dest_owner in df.dest_owner.unique()]
    counts = list(filter(lambda x: x!= 0, counts))
    cat_dict[category] = entropy(counts, base=None)
cat_entropy = pd.DataFrame(cat_dict.items(), columns = ['Category', 'Entropy'])
cat_entropy.sort_values(by=["Entropy"])
cat_entropy["CDN Count"] = [len(df[df.category == category].dest_owner.unique()) for category in cat_entropy['Category']]
cat_entropy["Top CDN"] = [df[df.category == category]['dest_owner'].value_counts().idxmax() for category in cat_entropy['Category']]
cat_entropy["# of Apps"] = [len(df[df.category == category].app.unique()) for category in cat_entropy['Category']]
print(cat_entropy)
    

/Users/lavanyasingh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


        Category   Entropy  CDN Count          Top CDN  # of Apps
0          games  1.536546         25  Comcast Cable C          2
1      lifestyle  1.914956         15  S - Akamai Tech          3
2       shopping  1.015942         12  Comcast Cable C          4
3         social  1.191922         18   Google LLC, US          4
4        finance  1.594803         14  2 - Amazon.com,          2
5  entertainment  0.677383          8  2 - Amazon.com,          1
6           kids  0.172813          4   Google LLC, US          2
7        control  0.339136          3   Google LLC, US          1
8           auto  0.007168          2   Google LLC, US          1


Keep in mind that the sample size for each category is pretty tiny.

Let's now try to figure out how distributed the entire dataset is. I'm going to use a a chi squared to test to figure out 
how evenly distributed CDN usage is.

In [58]:
from scipy.stats import chisquare
stat = chisquare([len(df[df["dest_owner"] == provider]) for provider in df.dest_owner.unique()])
print(stat.pvalue)

0.0


Our pvalue is 0. I guess CDN usage is nowhere NEAR unformly distributed. 

It also seems like "smaller" apps are more likely to use more CDN's than an app owned by a huge company, like Google. To test this, I'm going to use the app's downloads (sum of ios and android downloads as given by sensortower) as a rough proxy measure for app size (but maybe we can come up with something better). I'm going to have to manually input this :(

In [62]:
downloads = {"forgeahead":10000000, "tinder":6000000, "luckygo":1000000, "arrive":1800000, "yolo":2600000, 
              "venmo":1700000, "walmart":2000000, "cashapp":3000000, "tiktok":58000000, "messenger":43000000, 
             "amazonshopping":9000000, "roblox":12000000, "googlehome":7000000, "amazon":9000000, "disneyplus":8000000, 
             "messengerkids":900000, "baseline":0, "snapchat":26000000, "youtubekids":7000000, "androidauto":3000000}
df["downloads"] = [downloads[app] for app in df.app]
app_entropy["downloads"] = [downloads[app] for app in app_entropy.App]
app_entropy.sort_values(by=["downloads"])
print(app_entropy)


               App   Entropy  CDN Count          Top CDN       category  \
0       forgeahead  2.200191         23  S - Akamai Tech          games   
1           tinder  1.914604         10   Google LLC, US      lifestyle   
2          luckygo  1.560211         13  S - Akamai Tech      lifestyle   
3           arrive  1.425340          7   Google LLC, US       shopping   
4             yolo  1.368666         11   Google LLC, US         social   
5            venmo  1.367446         13  2 - Amazon.com,        finance   
6          walmart  1.158229          9  Comcast Cable C       shopping   
7          cashapp  1.127729          8   Google LLC, US        finance   
8           tiktok  1.021691          9          SN1, US         social   
9        messenger  0.950322         11   Google LLC, US         social   
10  amazonshopping  0.863552          5  Comcast Cable C       shopping   
11          roblox  0.750157          9  Comcast Cable C          games   
12      googlehome  0.697

Conclusions: There is definitely a lot of clustering going on, and sampling by category seems to be worth it because there's a good amount of variance in the entropy of each category. I think for future data collection we should also collect information about the size of the company that owns the app, the category of the app (so I don't have to input it manually lol), and the country of the app's owner (I didn't analyze that here, but TikTok talks to a Chinese CDN whereas the rest talk to American ones and that might be interesting).
